In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy as sp
from src.PLS.baseSIR import no_ext_sir
from src.PLS.baseSIR import timed_sir
from src.PLS.baseSIR import real_si
from decimal import Decimal, ROUND_UP
import time
import pandas as pd
from src.PLS.ABC import ABC_rejection
from src.PLS.ABC import ABC_core
from src.PLS.ABC import sum_sqrt_sq_distance

In [2]:
trajs = 10
iterations = 18

In [3]:
cluster_fittings ={}
cluster_fittings["1000"] = {}
cluster_fittings["100"] = {}
cluster_fittings["10"] = {}
#cluster_fittings["1000"][f"{i}"]

In [7]:
for i in range(1,trajs+1):
    total_traj = np.genfromtxt(f"../../Cluster/fittings/fit1000s/Traj_{i}_1_1000b__I1.csv",delimiter=",")
    for j in range(2,iterations +1):
        temp = np.genfromtxt(f"../../Cluster/fittings/fit1000s/Traj_{i}_{j}_1000b__I1.csv",delimiter=",")
        total_traj = np.append(temp,total_traj,axis=0)
    cluster_fittings["1000"][f"{i}"] = total_traj

In [ ]:
for i in range(1,3):
    total_traj = np.genfromtxt(f"../../Cluster/fittings/fit100s/Traj_{i}_1_100b.csv",delimiter=",")
    for j in range(2,iterations +1):
        temp = np.genfromtxt(f"../../Cluster/fittings/fit100s/Traj_{i}_{j}_100b.csv",delimiter=",")
        total_traj = np.append(temp,total_traj,axis=0)
    cluster_fittings["100"][f"{i}"] = total_traj

In [ ]:
for i in range(1,trajs+1):
    total_traj = np.genfromtxt(f"../../Cluster/fittings/fit10s/Traj_{i}_1_10b.csv",delimiter=",")
    print(".",end="")
    for j in range(2,iterations +1):
        temp = np.genfromtxt(f"../../Cluster/fittings/fit10s/Traj_{i}_{j}_10b.csv",delimiter=",")
        total_traj = np.append(temp,total_traj,axis=0)
    cluster_fittings["10"][f"{i}"] = total_traj

....

In [ ]:
def threshold_scheduler(results, threshold_references):
    sort_res = results.sort_values(by="Distance")
    output = []
    for i in threshold_references:
        temp = sort_res.iloc[0:int(len(sort_res.index) / (100 / i))]
        output.append(temp.iloc[-1, 1])
    return output

In [ ]:
graph_ABC = cluster_fittings["10"]["7"]
reference = [1]
df_ABC = pd.DataFrame(graph_ABC, columns=["Beta", "Distance"])
threshold_schedule = threshold_scheduler(df_ABC, reference)

In [ ]:
fig, ax0 = plt.subplots(figsize=(8, 8))
cols = sns.color_palette("rocket", n_colors=10)
for i in range(len(threshold_schedule)):
    thresh = ABC_rejection(threshold_schedule[i], graph_ABC)
    ploted = pd.DataFrame(thresh, columns=["Beta", "Counts"])
    bet_dis = ploted.groupby("Beta").count()
    plot_bet_dis = bet_dis[bet_dis["Counts"] != 0]
    max_scale = max(plot_bet_dis["Counts"])
    plot_bet_dis["Counts"] = plot_bet_dis["Counts"] / max_scale
    area = np.trapz(plot_bet_dis["Counts"],dx=0.01)
    plot_bet_dis["Counts"] = plot_bet_dis["Counts"] / area
    spline = sp.interpolate.CubicSpline(plot_bet_dis.index, plot_bet_dis["Counts"])
    print(spline)
    ax0.scatter(bet_dis.index, plot_bet_dis["Counts"], label=f"Threshold= {reference[i]}% accepted", color=cols[i],
                alpha=0.25)
    ax0.plot(plot_bet_dis.index, spline(plot_bet_dis.index), color=cols[i])
mles = np.genfromtxt("../../Data/R0_3/Fitted/MLE_b_I0_1_3.csv")
mle1 = mles[6]
ax0.set_ylabel("Posterior Probability Distribution")
ax0.set_xlabel("Beta")
ax0.axvline(mle1, color="red")
ax0.legend(bbox_to_anchor=(1, 1))
#ax0.set_ylim(0, 1.1)
#ax0.set_xlim(1,5)
fig.show()

In [ ]:
sample_cum = plot_bet_dis.copy()
sample_cum["Cum_prob"] = 0
for i in range(len(sample_cum.index)):
    sample_cum.iloc[i,1] = np.trapz(plot_bet_dis.iloc[0:i+1,0],dx=0.01)
print(sample_cum)

In [ ]:
fig, ax0 = plt.subplots()
ax0.plot(sample_cum.index,sample_cum["Cum_prob"])
fig.show()

In [ ]:
def beta_sampler(beta,cum_probs,rng,size):
    out = []
    rng = rng
    for i in range(size):
        copy_cums = cum_probs
        rn = np.random.uniform()
        filtered = copy_cums - rn
        filtered[filtered<0] = 1.1
        upper = np.argmin(filtered)
        lower = upper -1
        prob_distance = copy_cums[upper] - copy_cums[lower]
        beta_distance = beta[upper] - beta[lower]
        length_along_probs = (rn-copy_cums[lower])/prob_distance
        sampled_beta = beta[lower] + length_along_probs*beta_distance
        out.append(sampled_beta)
    return out

In [ ]:
rng = np.random.default_rng(1912)

In [ ]:
np.array(sample_cum.index)

In [ ]:
trial_sample = beta_sampler(np.array(sample_cum.index),np.array(sample_cum["Cum_prob"]),rng,100000)

In [ ]:
fig, ax0 = plt.subplots()
sns.histplot(trial_sample,ax=ax0)
ax0.plot(plot_bet_dis.index, spline(plot_bet_dis.index)*1400)
fig.show()